In [1]:
!pip install pandas==2.2.2 numpy pyspark==3.4.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 38.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285391 sha256=1c26ae3fa87a0697a41655cee791426a9a395bea1ca0530473606390f402d01f
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark
  Attempting uninstall: pandas
    Found existing installation: pandas 2.1.4
    Uninstalling pandas-2.1.4:
      Successfully uninstalled pandas-2.1.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 2.2.2 which is incompatible.
google-colab 1.0.0 requires pandas==2.1.4, but you have pandas 2.2.2 which is 

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('HandsOnPertemuan6').getOrCreate()

data = [('James', 'Sales', 3000),
        ('Michael', 'Sales', 4600),
        ('Robert', 'Sales', 4100),
        ('Maria', 'Finance', 3000)]
columns = ['EmployeeName', 'Department', 'Salary']

df = spark.createDataFrame(data, schema=columns)
df.show()

+------------+----------+------+
|EmployeeName|Department|Salary|
+------------+----------+------+
|       James|     Sales|  3000|
|     Michael|     Sales|  4600|
|      Robert|     Sales|  4100|
|       Maria|   Finance|  3000|
+------------+----------+------+



In [3]:
# Contoh operasi transformasi DataFrame
df.select('EmployeeName', 'Salary').show()
df.filter(df['Salary'] > 3000).show()
df.groupBy('Department').avg('Salary').show()

+------------+------+
|EmployeeName|Salary|
+------------+------+
|       James|  3000|
|     Michael|  4600|
|      Robert|  4100|
|       Maria|  3000|
+------------+------+

+------------+----------+------+
|EmployeeName|Department|Salary|
+------------+----------+------+
|     Michael|     Sales|  4600|
|      Robert|     Sales|  4100|
+------------+----------+------+

+----------+-----------+
|Department|avg(Salary)|
+----------+-----------+
|     Sales|     3900.0|
|   Finance|     3000.0|
+----------+-----------+



In [6]:
# Create a new DataFrame with the SalaryBonus column
df_with_bonus = df.withColumn('SalaryBonus', df['Salary'] * 0.1)

# Now create another DataFrame with the TotalCompensation column
df_with_total = df_with_bonus.withColumn('TotalCompensation', df_with_bonus['Salary'] + df_with_bonus['SalaryBonus'])

# Show the final DataFrame
df_with_total.show()


+------------+----------+------+-----------+-----------------+
|EmployeeName|Department|Salary|SalaryBonus|TotalCompensation|
+------------+----------+------+-----------+-----------------+
|       James|     Sales|  3000|      300.0|           3300.0|
|     Michael|     Sales|  4600|      460.0|           5060.0|
|      Robert|     Sales|  4100|      410.0|           4510.0|
|       Maria|   Finance|  3000|      300.0|           3300.0|
+------------+----------+------+-----------+-----------------+



In [7]:
# Contoh menggunakan window functions
from pyspark.sql.window import Window
from pyspark.sql import functions as F

windowSpec = Window.partitionBy('Department').orderBy('Salary')
df.withColumn('Rank', F.rank().over(windowSpec)).show()

+------------+----------+------+----+
|EmployeeName|Department|Salary|Rank|
+------------+----------+------+----+
|       Maria|   Finance|  3000|   1|
|       James|     Sales|  3000|   1|
|      Robert|     Sales|  4100|   2|
|     Michael|     Sales|  4600|   3|
+------------+----------+------+----+



In [ ]:
Ringkasan Operasi:
Pembuatan DataFrame Dasar:
Membuat DataFrame Spark dari daftar tuple.
Ini termasuk data dasar seperti EmployeeName, Department, dan Salary.
Transformasi Dasar:

Seleksi dan Penyaringan:
Memilih kolom tertentu dari DataFrame.
Menyaring baris di mana Salary lebih besar dari nilai tertentu.

Agregasi:
Menggunakan groupBy untuk menghitung rata-rata gaji untuk setiap departemen.

Tipe Data Kompleks:
Operasi Kolom:
Menambahkan kolom baru seperti SalaryBonus (10% dari Salary).
Membuat kolom lain TotalCompensation, yang merupakan jumlah dari Salary dan SalaryBonus.

Operasi Lanjutan:
Window Functions:
Menggunakan window functions untuk memberikan peringkat kepada karyawan berdasarkan gaji mereka di dalam departemen.
Hal ini memungkinkan perhitungan peringkat, total berjalan, dan komputasi terpartisi.

Bagaimana Teknik Ini Dapat Diterapkan pada Proyek Nyata:
Transformasi Data:
Kemampuan untuk mentransformasi data secara efisien (penyaringan, seleksi, dan penambahan kolom) sangat penting untuk pembersihan dan persiapan data dalam proyek seperti analisis keuangan atau segmentasi pelanggan.

Agregasi dan Pengelompokan:
Teknik agregasi (misalnya, groupBy, avg) sangat penting dalam menghasilkan wawasan dari dataset besar dalam analitik real-time atau sistem pelaporan.

Penanganan Tipe Data Kompleks:
Memanipulasi tipe data kompleks memungkinkan pemodelan data yang lebih rinci, seperti menangani data hierarkis atau bersarang dalam format seperti JSON, yang sering ditemukan dalam aliran data web atau IoT.

Window Functions:
Window functions sangat kuat dalam analisis deret waktu, untuk menghitung rata-rata bergerak, total berjalan, atau peringkat dalam dasbor intelijen bisnis.